## 2. Working with Data in Tableau
----
Tableau에서 다양한 방법으로 데이터를 불러오고 필터를 통해 원하는 데이터를 추출하는 방법에 대해 공부하였습니다.

* CSV, 엑셀파일, Cloud 내 데이터, Server 등 으로부터 데이터를 불러 올 수 있다.
* Tableau에서는 Visual Query Language(VizQL)를 이용하여 데이터를 원하는 형태로 바꿀 수 있다
* Database에 직접 연결하여 데이터를 불러올 수 있으나 책의 예제는 없었다. 필요 시 따로 더 찾아볼 필요성 존재.
* 온라인 Tableau는 데스크탑용 Tableau에 반해 기능이 다수 빠져있음을 확인하였다.

## Regression 데이터 재선정 논의
----

자료 조사 결과 시계열을 제외해야하기 때문에 횡단자료 (cross-sectional data)가 필요함을 느꼈습니다. Kaggle이나 UCI에서는 회귀데이터의 수가 적으며, 있어도 사전지식을 파악하기가 어려웠습니다. 이에 국내 데이터를 찾았고, 질병관리본부에서 제공하는 국민건강영양조사 데이터를 이용해보기로 하였습니다.

### Objective of the notebook
----
나이가 들면 자연적으로 근육이 줄고 근력도 떨어진다는 생각에 질병으로 인식되지 않던 근감소증이 노년기 건강악화의 지표가 될 수 있다는 연구 결과가 나왔다. 근감소증을 진단하는 도구로는 근력을 측정하는 방법과 신체기능평가 방법을 이용하여 진단하고 있다. 근력측정은 dual energy X-ray absorptiometry (DXA)를 이용하여 측정된 사지근육량을 키의 제곱으로 나눈 $ASM / height^2$을 이용하여 기준값을 이용하여 진단하며, 추가적으로 악력을 사용하여 진단하기도 한다. 신체기능평가는 보행속도를 측정하여 초당 m가(m/s)가 0.8 이하로 구분하여 평가한다.

이번 연구에서는 근감소증을 진단하는 요소 중 하나로 악력을 예측하고자 한다. 이에 국민건강영양조사 자료를 탐색하여, 대상의 기본정보, 지병여부, 영양소섭취량 정도가 악력에 영향을 미치는지 알아보고자 한다.

![악력기](strength_dynamometer.png)

### About the dataset : 국민건강영양조사
----
국민건강영양조사의 표본 추출틀은 표본설계 시점에서 가용한 가장 최근 시점의 인구주택총조사 자료를 사용하였고, 이를 통해 목표 모집단인 대한민국에 거주하는 만1세이상 국민에 대하여 대표성 있는 표본을 추출할 수 있도록 하였다. 표본추출방법은 조사구, 가구를 1,2차 추출단위로 하는 2단계 층화집락표본추출방법을 사용하였다.

건강설문조사와 검진조사는 이동검진센터에서 실시하였으며 영양조사는 대상 가구를 직접 방문하여 실시하였다. 건강설문조사의 교육 및 경제활동, 이환, 의료이용 항목, 영양조사의 전체 항목은 면접방법으로 조사하였으며, 건강설문조사 항목 중 흡연, 음주 등 건강행태 영역은 자기기입식으로 조사하였다.

국민건강영양조사 자료는 총 48개의 속성으로 이루어져 있으며, **오른손 최대악력과 왼손 최대악력 중 최대값**(개개인마다 주로 이용하는 손이 차이나기 때문)을 Target으로 하고자 한다. 

48개의 자료 필드는 아래 표와 같다. 

|num|변수|유형|길이|레이블|항목값|조사도구|척도|
|:-:|:---|---|---|:----|:----|:-------|---|
|1|ID|문자|10|개인 아이디|||명목(key)|
|2|sex|숫자|8|성별|1. 남자<BR>2. 여자|설문조사|명목|
|3|age|숫자|8|만나이|1~79. 1~79세<BR>80. 80세이상|설문조사|순서+명목(80세이상)|
|4|ho_incm|숫자|8|소득4분위수(가구)|1. 하<BR>2. 중하<BR>3. 중상<BR>4. 상|설문조사|순서|
|5|edu|숫자|8|교육수준 재분류 코드|1. 초졸이하<BR>2. 중졸<BR>3. 고졸<BR>4. 대졸이상|설문조사|순서|
|6|occp|숫자|8|직업재분류 및 실업/비경제활동인구상태 코드|1. 관리자, 전문가 및 관련 종사자<BR>2. 사무종사자<BR>3. 서비스 및 판매 종사자<BR>4. 농림어업 숙련 종사자<BR>5. 기능원, 장치․기계조작 및 조립종사자<BR>6. 단순노무종사자<BR>7. 무직(주부, 학생 등)|설문조사|명목|
|7|marri_1|숫자|8|결혼여부|1. 기혼<BR>2. 미혼<BR>9. 모름, 무응답|설문조사|명목|
|8|sm_presnt|숫자|8|현재흡연율|0. 아니오<BR>1. 예|설문조사|명목, 더미|
|9|dr_high|숫자|8|고위험음주여부|0. 아니오<BR>1. 예|설문조사|명목, 더미|
|10|pa_aerobic|숫자|8|유산소신체활동실천여부|0. 아니오<BR>1. 예|설문조사|명목, 더미|
|11|pa_walk|숫자|8|걷기실천여부|0. 아니오<BR>1. 예|설문조사|명목, 더미|
|12|pa_muscle|숫자|8|근력운동여부|0. 아니오<BR>1. 예|설문조사|명목, 더미|
|13|HE_ht|숫자|8|신장|□□□.□cm|검진조사|비율|
|14|HE_wt|숫자|8|체중|□□□.□ kg|검진조사|비율|
|15|HE_wc|숫자|8|허리둘레|□□□.□ cm|검진조사|비율|
|16|HE_BMI|숫자|8|체질량지수|□□□.□ kg/m2|검진조사|비율|
|17|HE_HP|숫자|8|고혈압 유병여부(19세이상)|1. 정상<BR>2. 고혈압전단계<BR>3.고혈압|수축기혈압이 140이상이거나 이완기혈압이 90이상인 여부<BR>*고혈압 진단기준|명목, 더미|
|18|HE_DM|숫자|8|당뇨병 유병여부(19세이상)|1. 정상<BR>2. 공복혈당장애<BR>3.당뇨병|공복혈당≥ 126 또는 당뇨병약 복용 또는 인슐린주사 투여 또는 의사진단자|명목, 더미|
|19|HE_HCHOL|숫자|8|고콜레스테롤혈증 유병여부(19세이상)|0. 아니오<BR>1. 예|공복시 총콜레스테롤≥240㎎/㎗ 또는 콜레스테롤약 복용|명목, 더미|
|20|GS_use|숫자|8|주로 사용하는 손|1. 오른손<BR>2. 왼손<BR>3. 양손<BR>8. 비해당(10세미만)<BR>9. 무응답(미검진)||명목|
|21|N_WAT_C|숫자|8|물섭취량(컵)|물섭취량(컵: 200ml)|24시간 회상법으로 조사한 식품섭취조사의 식품을 환산한 결과<BR>*자세한 내용은 국민건강영양조사(https://knhanes.cdc.go.kr/knhanes/main.do)에서 확인가능|비율|
|22|N_INTK|숫자|8|식품섭취량(g)|식품별 섭취량(g)||비율|
|23|N_EN|숫자|8|에너지섭취량(Kcal)|에너지섭취량(Kcal)||비율|
|24|N_WATER|숫자|8|수분섭취량(g)|수분섭취량(g)||비율|
|25|N_PROT|숫자|8|단백질섭취량(g)|단백질섭취량(g)||비율|
|26|N_FAT|숫자|8|지방섭취량(g)|지방섭취량(g)||비율|
|27|N_SFA|숫자|8|포화지방산 섭취량(g)|포화지방산 섭취량(g)||비율|
|28|N_MUFA|숫자|8|단일불포화지방산 섭취량(g)|단일불포화지방산 섭취량(g)||비율|
|29|N_PUFA|숫자|8|다가불포화지방산 섭취량(g)|다가불포화지방산 섭취량(g)||비율|
|30|N_N3|숫자|8|n-3계 지방산 섭취량(g)|n-3계 지방산 섭취량(g)||비율|
|31|N_N6|숫자|8|n-6계 지방산 섭취량(g)|n-6계 지방산 섭취량(g)||비율|
|32|N_chol|숫자|8|콜레스테롤섭취량(mg)|콜레스테롤섭취량(mg)||비율|
|33|N_CHO|숫자|8|탄수화물섭취량(g)|탄수화물섭취량(g)||비율|
|34|N_tdf|숫자|8|식이섬유섭취량(g)|식이섬유섭취량(g)||비율|
|35|N_CA|숫자|8|칼슘섭취량(mg)|칼슘섭취량(mg)||비율|
|36|N_PHOS|숫자|8|인섭취량(mg)|인섭취량(mg)||비율|
|37|N_FE|숫자|8|철섭취량(mg)|철섭취량(mg)||비율|
|38|N_NA|숫자|8|나트륨섭취량(mg)|나트륨섭취량(mg)||비율|
|39|N_K|숫자|8|칼륨섭취량(mg)|칼륨섭취량(mg)||비율|
|40|N_VA|숫자|8|비타민A섭취량(μgRE)|비타민A섭취량(μgRE)||비율|
|41|N_CAROT|숫자|8|카로틴섭취량(μg)|카로틴섭취량(μg)||비율|
|42|N_RETIN|숫자|8|레티놀섭취량(μg)|레티놀섭취량(μg)||비율|
|43|N_B1|숫자|8|티아민섭취량(mg)|티아민섭취량(mg)||비율|
|44|N_B2|숫자|8|리보플라빈섭취량(mg)|리보플라빈섭취량(mg)||비율|
|45|N_NIAC|숫자|8|나이아신섭취량(mg)|나이아신섭취량(mg)||비율|
|46|N_VITC|숫자|8|비타민C섭취량(mg)|비타민C섭취량(mg)||비율|
|47|max_grip_rt|숫자|8|오른손 최대악력|1~3차 악력측정값 중 최대값|타겟변수는 양손 최대악력중 최대값으로 사용|비율|
|48|max_grip_lt|숫자|8|왼손 최대악력|||비율|


In [30]:
# 자료 불러오기
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_sas('grip_str.sas7bdat').dropna() 

In [31]:
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,max_grip_rt,max_grip_lt
0,b'A209716213',2.0,30.0,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,3035.899499,1335.998584,7476.633795,88.505252,2.524458,1.702372,13.792368,69.967986,28.4,26.4
2,b'A209716216',1.0,29.0,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,5018.153615,2181.289291,11339.248496,291.693538,3.853337,3.065095,22.947486,65.287764,50.6,47.8
3,b'A209724716',1.0,65.0,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,1952.808521,452.073639,2102.115596,100.982500,1.164780,1.098967,11.789950,43.467542,33.1,33.6
4,b'A209731516',1.0,66.0,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,2339.124348,854.972445,4817.173801,51.367319,1.015765,0.579514,10.199538,56.882524,36.5,35.4
6,b'A209736615',2.0,41.0,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,2214.850076,276.389343,1509.860228,19.627705,0.932423,0.742740,16.719703,146.408221,31.7,29.6


In [32]:
df.age.max()

80.0

In [33]:
df.age.min()

19.0

In [34]:
labels = ["{0} - {1}".format(i, i + 10) for i in range(19, 80, 10)]

In [35]:
labels

['19 - 29', '29 - 39', '39 - 49', '49 - 59', '59 - 69', '69 - 79', '79 - 89']

In [36]:
df['age']=df.age.astype('int64')

In [37]:
df['age'] = pd.cut(df.age, range(19, 90, 10), right=False, labels=labels)

In [38]:
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,max_grip_rt,max_grip_lt
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,3035.899499,1335.998584,7476.633795,88.505252,2.524458,1.702372,13.792368,69.967986,28.4,26.4
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,5018.153615,2181.289291,11339.248496,291.693538,3.853337,3.065095,22.947486,65.287764,50.6,47.8
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,1952.808521,452.073639,2102.115596,100.982500,1.164780,1.098967,11.789950,43.467542,33.1,33.6
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,2339.124348,854.972445,4817.173801,51.367319,1.015765,0.579514,10.199538,56.882524,36.5,35.4
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,2214.850076,276.389343,1509.860228,19.627705,0.932423,0.742740,16.719703,146.408221,31.7,29.6


In [39]:
df.dtypes

ID               object
sex             float64
age            category
ho_incm         float64
edu             float64
occp            float64
marri_1         float64
sm_presnt       float64
dr_high         float64
pa_aerobic      float64
pa_walk         float64
pa_muscle       float64
HE_ht           float64
HE_wt           float64
HE_wc           float64
HE_BMI          float64
HE_HP           float64
HE_DM           float64
HE_HCHOL        float64
GS_use          float64
N_WAT_C         float64
N_INTK          float64
N_EN            float64
N_WATER         float64
N_PROT          float64
N_FAT           float64
N_SFA           float64
N_MUFA          float64
N_PUFA          float64
N_N3            float64
N_N6            float64
N_chol          float64
N_CHO           float64
N_tdf           float64
N_CA            float64
N_PHOS          float64
N_FE            float64
N_NA            float64
N_K             float64
N_VA            float64
N_CAROT         float64
N_RETIN         

In [40]:
for i in ['sex','age']:
    df=pd.concat([df,pd.get_dummies(df[i], prefix=i)],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,max_grip_lt,sex_1.0,sex_2.0,age_19 - 29,age_29 - 39,age_39 - 49,age_49 - 59,age_59 - 69,age_69 - 79,age_79 - 89
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,26.4,0,1,0,1,0,0,0,0,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,47.8,1,0,0,1,0,0,0,0,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,33.6,1,0,0,0,0,0,1,0,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,35.4,1,0,0,0,0,0,1,0,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,29.6,0,1,0,0,1,0,0,0,0


In [41]:
df=pd.concat([df,pd.get_dummies(df.ho_incm, prefix='ho_incm')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,age_29 - 39,age_39 - 49,age_49 - 59,age_59 - 69,age_69 - 79,age_79 - 89,ho_incm_1.0,ho_incm_2.0,ho_incm_3.0,ho_incm_4.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,0,1,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,0,1,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,1,0,0,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,0,1,0,0,0,1,0,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,1,0,0,0,0,0,0,1,0


In [42]:
df=pd.concat([df,pd.get_dummies(df.edu, prefix='edu')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,age_69 - 79,age_79 - 89,ho_incm_1.0,ho_incm_2.0,ho_incm_3.0,ho_incm_4.0,edu_1.0,edu_2.0,edu_3.0,edu_4.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,1
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,1,0,0,0,1,0,0,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,1
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,0,0,0,1,0,0,0,1,0


In [43]:
df=pd.concat([df,pd.get_dummies(df.occp, prefix='occp')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,edu_2.0,edu_3.0,edu_4.0,occp_1.0,occp_2.0,occp_3.0,occp_4.0,occp_5.0,occp_6.0,occp_7.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,1,0,0,0,0,0,0,1
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,1,0,0,0,0,0,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,1,0,0,0,0,0,0,1
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,1,0,0,0,0,0,0,0,1


In [44]:
df=pd.concat([df,pd.get_dummies(df.marri_1, prefix='marri_1')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,edu_4.0,occp_1.0,occp_2.0,occp_3.0,occp_4.0,occp_5.0,occp_6.0,occp_7.0,marri_1_1.0,marri_1_2.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,1,0,1
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,1,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,0,0,1,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,1,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,1,1,0


In [45]:
df=pd.concat([df,pd.get_dummies(df.sm_presnt, prefix='sm_presnt')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,occp_2.0,occp_3.0,occp_4.0,occp_5.0,occp_6.0,occp_7.0,marri_1_1.0,marri_1_2.0,sm_presnt_0.0,sm_presnt_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,1,1,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,1,1,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,1,0,0,0,0,1,0,0,1
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,1,1,0,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,1,1,0,1,0


In [46]:
df=pd.concat([df,pd.get_dummies(df.dr_high, prefix='dr_hight')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,occp_4.0,occp_5.0,occp_6.0,occp_7.0,marri_1_1.0,marri_1_2.0,sm_presnt_0.0,sm_presnt_1.0,dr_hight_0.0,dr_hight_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,1,0,1,1,0,1,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,0,1,0,1,1,0,1,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,0,0,1,0,0,1,1,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,0,1,1,0,1,0,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,0,0,1,1,0,1,0,0,1


In [47]:
df=pd.concat([df,pd.get_dummies(df.pa_aerobic, prefix='pa_aerobic')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,occp_6.0,occp_7.0,marri_1_1.0,marri_1_2.0,sm_presnt_0.0,sm_presnt_1.0,dr_hight_0.0,dr_hight_1.0,pa_aerobic_0.0,pa_aerobic_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,0,1,1,0,1,0,0,1
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,0,1,1,0,1,0,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,1,0,0,1,1,0,0,1
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,1,1,0,1,0,1,0,0,1
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,1,1,0,1,0,0,1,0,1


In [48]:
df=pd.concat([df,pd.get_dummies(df.pa_walk, prefix='pa_walk')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,marri_1_1.0,marri_1_2.0,sm_presnt_0.0,sm_presnt_1.0,dr_hight_0.0,dr_hight_1.0,pa_aerobic_0.0,pa_aerobic_1.0,pa_walk_0.0,pa_walk_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,1,0,1,0,0,1,1,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,1,0,1,0,0,1,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,1,0,0,1,1,0,0,1,0,1
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,1,0,1,0,0,1,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,1,0,1,0,0,1,0,1,0,1


In [49]:
df=pd.concat([df,pd.get_dummies(df.pa_muscle, prefix='pa_muscle')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,sm_presnt_0.0,sm_presnt_1.0,dr_hight_0.0,dr_hight_1.0,pa_aerobic_0.0,pa_aerobic_1.0,pa_walk_0.0,pa_walk_1.0,pa_muscle_0.0,pa_muscle_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,1,0,0,1,1,0,0,1
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,1,0,0,1,0,1,0,1
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,1,1,0,0,1,0,1,1,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,1,0,0,1,1,0,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,1,0,0,1,0,1,0,1,1,0


In [50]:
df=pd.concat([df,pd.get_dummies(df.HE_HP, prefix='HE_HP')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,dr_hight_1.0,pa_aerobic_0.0,pa_aerobic_1.0,pa_walk_0.0,pa_walk_1.0,pa_muscle_0.0,pa_muscle_1.0,HE_HP_1.0,HE_HP_2.0,HE_HP_3.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,1,1,0,0,1,1,0,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,1,0,1,0,1,0,1,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,0,1,0,1,1,0,0,0,1
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,0,0,1,1,0,1,0,0,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,1,0,1,0,1,1,0,0,1,0


In [51]:
df=pd.concat([df,pd.get_dummies(df.HE_DM, prefix='HE_DM')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,pa_walk_0.0,pa_walk_1.0,pa_muscle_0.0,pa_muscle_1.0,HE_HP_1.0,HE_HP_2.0,HE_HP_3.0,HE_DM_1.0,HE_DM_2.0,HE_DM_3.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,0,1,1,0,0,1,0,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,0,1,0,1,0,1,0,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,1,1,0,0,0,1,0,0,1
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,1,0,0,1,0,1,0,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,0,1,1,0,0,1,0,1,0,0


In [52]:
df=pd.concat([df,pd.get_dummies(df.HE_HCHOL, prefix='HE_HCHOL')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,pa_muscle_0.0,pa_muscle_1.0,HE_HP_1.0,HE_HP_2.0,HE_HP_3.0,HE_DM_1.0,HE_DM_2.0,HE_DM_3.0,HE_HCHOL_0.0,HE_HCHOL_1.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,1,0,0,1,0,0,1,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,1,0,1,0,1,0,0,1,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,1,0,0,0,1,0,0,1,1,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,0,1,0,1,0,0,1,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,1,0,0,1,0,1,0,0,1,0


In [53]:
df=pd.concat([df,pd.get_dummies(df.GS_use, prefix='GS_use')],axis=1)
df.head()

,ID,sex,age,ho_incm,edu,occp,marri_1,sm_presnt,dr_high,pa_aerobic,...,HE_HP_2.0,HE_HP_3.0,HE_DM_1.0,HE_DM_2.0,HE_DM_3.0,HE_HCHOL_0.0,HE_HCHOL_1.0,GS_use_1.0,GS_use_2.0,GS_use_3.0
0,b'A209716213',2.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,0,0,1,0,0,1,0,1,0,0
2,b'A209716216',1.0,29 - 39,3.0,4.0,7.0,2.0,0.0,0.0,1.0,...,1,0,1,0,0,1,0,1,0,0
3,b'A209724716',1.0,59 - 69,1.0,1.0,3.0,1.0,1.0,0.0,1.0,...,0,1,0,0,1,1,0,1,0,0
4,b'A209731516',1.0,59 - 69,2.0,4.0,7.0,1.0,0.0,0.0,1.0,...,1,0,1,0,0,1,0,1,0,0
6,b'A209736615',2.0,39 - 49,3.0,3.0,7.0,1.0,0.0,1.0,1.0,...,1,0,1,0,0,1,0,1,0,0


In [54]:
df=df.drop(['sex','age','ho_incm','edu','occp','marri_1','sm_presnt','dr_high','pa_aerobic','pa_walk','pa_muscle','HE_HP','HE_DM','HE_HCHOL','GS_use'],axis=1)

In [55]:
df.head()

,ID,HE_ht,HE_wt,HE_wc,HE_BMI,N_WAT_C,N_INTK,N_EN,N_WATER,N_PROT,...,HE_HP_2.0,HE_HP_3.0,HE_DM_1.0,HE_DM_2.0,HE_DM_3.0,HE_HCHOL_0.0,HE_HCHOL_1.0,GS_use_1.0,GS_use_2.0,GS_use_3.0
0,b'A209716213',153.4,68.1,81.9,28.939858,3.5,1198.226616,1930.920992,761.873294,69.515824,...,0,0,1,0,0,1,0,1,0,0
2,b'A209716216',171.2,84.0,85.5,28.659708,4.0,2099.617699,3007.329713,1427.204406,109.849807,...,1,0,1,0,0,1,0,1,0,0
3,b'A209724716',160.8,52.2,71.0,20.188238,3.0,779.230888,1502.406756,443.763134,57.129583,...,0,1,0,0,1,1,0,1,0,0
4,b'A209731516',168.8,68.8,84.0,24.145909,5.0,1165.590400,1116.380721,911.584789,36.183575,...,1,0,1,0,0,1,0,1,0,0
6,b'A209736615',162.2,80.5,98.4,30.598080,15.0,2336.472416,2465.075672,1897.949605,72.131360,...,1,0,1,0,0,1,0,1,0,0


In [56]:
df.shape

(11323, 80)

In [57]:
# Target 컬럼 생성 - 왼손 악력과 오른손 악력 중 최대값을 타겟으로 한다.
target = df[['max_grip_rt', 'max_grip_lt']].max(axis = 1)
features = df.drop(columns = ['ID', 'max_grip_rt', 'max_grip_lt'], axis = 1)

In [58]:
# Train데이터와 Test를 나눈다.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=0)

In [59]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

## 질문할 것 : 범주형 변수가 있을 때 전처리를 하면 어떻게되는건지?

## DecisionTreeRegressor, ExtraTreeRegressor 중에 어떤 것이 성능이 좋은지 테스트해봤습니다.

In [60]:
dtr=DecisionTreeRegressor()

In [61]:
dtr.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [62]:
dtr.score(X_test,y_test)

0.47642908478078416

In [63]:
exr=ExtraTreeRegressor()

In [64]:
exr.fit(X_train, y_train)

ExtraTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random')

In [65]:
exr.score(X_test, y_test)

0.4990827600907193

In [66]:
tree_std=Pipeline([('scaler',StandardScaler()),('exr',ExtraTreeRegressor())])

In [67]:
tree_std.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('exr', ExtraTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'))])

In [68]:
tree_std.score(X_test,y_test)

0.4684384072455808

In [69]:
tree_mm=Pipeline([('scaler',MinMaxScaler()),('exr',ExtraTreeRegressor())])

In [70]:
tree_mm.fit(X_train,y_train)
tree_mm.score(X_test,y_test)

0.4658349528565716

## 그리드서치를 통해 최적의 하이퍼파라미터를 찾고 결과를 확인합니다.
## 정확도가 많이 향상된 것을 확인할 수 있습니다.

In [71]:
params = {'max_depth': np.arange(5, 21,5),'random_state':np.arange(1,20)}

In [72]:
tree_grid=GridSearchCV(ExtraTreeRegressor(), params)

In [73]:
tree_grid.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=ExtraTreeRegressor(criterion='mse', max_depth=None, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=None,
          splitter='random'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 5, 10, 15, 20]), 'random_state': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [74]:
tree_grid.best_params_

{'max_depth': 5, 'random_state': 17}

In [75]:
tree_grid.best_score_

0.7233712857338827

In [76]:
tree=ExtraTreeRegressor(**tree_grid.best_params_)

In [77]:
tree.fit(X_train,y_train)

ExtraTreeRegressor(criterion='mse', max_depth=5, max_features='auto',
          max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, random_state=17, splitter='random')

In [78]:
tree.score(X_test,y_test)

0.7131715120978847

## KNeighborsRegressor

In [79]:
knr=KNeighborsRegressor()

In [80]:
knr.fit(X_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [81]:
knr.score(X_test,y_test)

-0.0012476775745915436

## 전처리결과 MinMaxScaler의 결과가 좋음을 확인했습니다.
## 질문 : 그리드서치할 때 전처리 한 데이터를 넣어줘야하나?

In [82]:
knn_std=Pipeline([('scaler',StandardScaler()),('exr',KNeighborsRegressor())])

In [83]:
knn_std.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('exr', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'))])

In [84]:
knn_std.score(X_test,y_test)

0.6208255593727382

In [85]:
knn_min=Pipeline([('scaler',MinMaxScaler()),('exr',KNeighborsRegressor())])

In [86]:
knn_min.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('exr', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'))])

In [87]:
knn_min.score(X_test,y_test)

0.657833598438195

In [88]:
params = {'n_neighbors': np.arange(1, 20,5),'algorithm':['ball_tree', 'kd_tree'],'leaf_size':np.arange(20,40,5)}

In [89]:
knn_grid=GridSearchCV(KNeighborsRegressor(), params)

In [90]:
knn_grid.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  6, 11, 16]), 'algorithm': ['ball_tree', 'kd_tree'], 'leaf_size': array([20, 25, 30, 35])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [91]:
knn_grid.best_params_

{'algorithm': 'ball_tree', 'leaf_size': 20, 'n_neighbors': 16}

In [92]:
knn_grid.best_score_

0.11242548311758481

In [93]:
knn=Pipeline([('scaler',MinMaxScaler()),('tree',KNeighborsRegressor(**knn_grid.best_params_))])

In [94]:
knn.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('tree', KNeighborsRegressor(algorithm='ball_tree', leaf_size=20, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=16, p=2,
          weights='uniform'))])

In [95]:
knn.score(X_test,y_test)

0.6944978911820527